In [1]:
import sympy
from sympy import *
from sympy.tensor.array import MutableSparseNDimArray as n_array
import numpy as np

In [2]:
num_vars = 4 # can set to an arbitrary dimension

t,r,θ,φ = symbols('t r θ φ')
vars = [t,r,θ,φ] # REMEMBER TO CHANGE THIS

a = Function('a')(r,t)
b = Function('b')(r,t)

In [3]:
# Input the metric here; Variables can be entered by indexing the array "x". The time t can be entered simply using "t"

metric = Matrix([[-exp(a),0,0,0],
                [0,exp(b),0,0],
                [0,0,r**2,0],
                [0,0,0,r**2 * sin(θ)**2]])

metric_inv = metric.inv() # Take inverse of metric

christoffel_symbols = n_array([], shape=(num_vars,num_vars,num_vars))
for lambda_ in range(len(vars)):
    for mu in range(len(vars)):
        for nu in range(len(vars)):
            summed = 0
            for sigma in range(len(vars)):
                summed += 0.5 * metric_inv[lambda_, sigma] * (simplify(diff(metric[nu,sigma], vars[mu])) + simplify(diff(metric[sigma, mu], vars[nu])) - simplify(diff(metric[mu, nu], vars[sigma])))
            christoffel_symbols[lambda_, mu, nu] = simplify(summed)

riemann_tensor = n_array([], shape=(num_vars,num_vars,num_vars,num_vars))
for rho in range(len(vars)):
    for sigma in range(len(vars)):
        for mu in range(len(vars)):
            for nu in range(len(vars)):
                summed_component_1 = 0
                summed_component_2 = 0
                
                for lambda_ in range(len(vars)):
                    summed_component_1 += simplify(christoffel_symbols[rho, mu, lambda_] * christoffel_symbols[lambda_, nu, sigma])
                    summed_component_2 += simplify(christoffel_symbols[rho, nu, lambda_] * christoffel_symbols[lambda_, mu, sigma])
                riemann_tensor[rho,sigma,mu,nu] = simplify(simplify(diff(christoffel_symbols[rho, nu, sigma], vars[mu])) - simplify(diff(christoffel_symbols[rho, mu, sigma], vars[nu])) + simplify(summed_component_1) - simplify(summed_component_2)) 

ricci_tensor = n_array([], shape=(num_vars, num_vars))
for mu in range(len(vars)):
    for nu in range(len(vars)):
        for lambda_ in range(len(vars)):
            ricci_tensor[mu, nu] += riemann_tensor[lambda_, mu, lambda_, nu]
            ricci_tensor[mu, nu] = simplify(ricci_tensor[mu,nu])

ricci_scalar = 0
for mu in range(len(vars)):
    for nu in range(len(vars)):
        ricci_scalar += ricci_tensor[mu,nu] * metric_inv[mu,nu]
ricci_scalar = simplify(ricci_scalar)

einstein_tensor = n_array([], (num_vars, num_vars))
for mu in range(len(vars)):
    for nu in range(len(vars)):
        einstein_tensor[mu,nu] = factor(cancel(expand(ricci_tensor[mu,nu] - 0.5 * ricci_scalar * metric[mu,nu])))


In [4]:
einstein_tensor

[[1.0*(r*Derivative(b(r, t), r) + exp(b(r, t)) - 1)*exp(a(r, t))*exp(-b(r, t))/r**2, 1.0*Derivative(b(r, t), t)/r, 0, 0], [1.0*Derivative(b(r, t), t)/r, 1.0*(r*Derivative(a(r, t), r) - exp(b(r, t)) + 1)/r**2, 0, 0], [0, 0, 0.5*r*(0.5*r*exp(a(r, t))*Derivative(a(r, t), r)**2 - 0.5*r*exp(a(r, t))*Derivative(a(r, t), r)*Derivative(b(r, t), r) + 1.0*r*exp(a(r, t))*Derivative(a(r, t), (r, 2)) + 0.5*r*exp(b(r, t))*Derivative(a(r, t), t)*Derivative(b(r, t), t) - 0.5*r*exp(b(r, t))*Derivative(b(r, t), t)**2 - 1.0*r*exp(b(r, t))*Derivative(b(r, t), (t, 2)) + 1.0*exp(a(r, t))*Derivative(a(r, t), r) - 1.0*exp(a(r, t))*Derivative(b(r, t), r))*exp(-a(r, t))*exp(-b(r, t)), 0], [0, 0, 0, 0.5*r*(0.5*r*exp(a(r, t))*Derivative(a(r, t), r)**2 - 0.5*r*exp(a(r, t))*Derivative(a(r, t), r)*Derivative(b(r, t), r) + 1.0*r*exp(a(r, t))*Derivative(a(r, t), (r, 2)) + 0.5*r*exp(b(r, t))*Derivative(a(r, t), t)*Derivative(b(r, t), t) - 0.5*r*exp(b(r, t))*Derivative(b(r, t), t)**2 - 1.0*r*exp(b(r, t))*Derivative(b(r,